# Homework 1

In this notebook we explore some algorithms to solve nonlinear equations.

Our objective is to find x so that $\sin x = 0.2$. Since the sine function is periodic, we will look for $x$ in the interval $[0,\pi/2]$.

Let us start by defining the function $f(x) = \sin(x) - 0.2$. We want to find $x \in [0,\pi/2]$ so that $f(x) = 0$. We also define the derivative of $f$, because we need it for Newton's method.

In [1]:
function f(x)
    return sin(x) - 0.2
end

f (generic function with 1 method)

In [2]:
function df(x)
    return cos(x)
end

df (generic function with 1 method)

Actually, we can find the right value of $x$ without much work by applying the built-in asin function. Let's do it just to make sure our code below works well.

In [3]:
x_exact = asin(0.2)

0.2013579207903308

The following code implements Newton's method. We discussed it in class. I also included the variable `counter` here, that keeps track of the number of iterations. It gets printed out at the end. If we iterate too many times, the program stops and returns nothing.

In the code below, we assign a default value 1e-20 to the variable `tolerance`. That means that whenever we call the `newton` function later, its 4th parameter is optional.

In [4]:
function newton(f::Function,df::Function,x0::Real,tolerance::Real=1e-15)
    x = x0
    counter = 0
    while abs(f(x)) > tolerance
        x = x - f(x) / df(x)
        counter = counter + 1
        if counter > 1000
            println("We give up!")
            return
        end
    end
    println("We performed ", counter, " iterations.")
    return x
end

newton (generic function with 2 methods)

Let's try it. I guess that if we start with $x_0 = \pi/4$ the sequence should converge to the right value.

In [5]:
x_newton = newton(f,df,pi/4)

We performed 5 iterations.


0.2013579207903308

We got the right value after very few iterations. It works very well in this case. Note that we had to define the derivative of the function beforehand, which we may not always be able to do. We also had to guess a good initial point for the interation.

Let us implement the secant method now. That's up to you.

In [6]:
function secant(f::Function,x0::Real,x1::Real,tolerance::Real=1e-15)
    counter = 0
    while abs(f(x1)) > tolerance
        # Replace the following line with the right formula for x2.
        x2 = (x1*f(x0)-x0*f(x1))/(f(x0)-f(x1))
        x0 = x1
        x1 = x2
        counter = counter + 1
        if counter > 1000
            println("We give up!")
            return
        end
    end
     println("We performed ", counter, " iterations.")
    return x1
end
    

secant (generic function with 2 methods)

In [7]:
x_secant = secant(f,0,pi/2)

We performed 6 iterations.


0.20135792079033143

It seems to be working fairly well as well.

Let us implement a couple of other methods. First, we will do a silly binary search. We notice that since we know that $\sin 0 = 0 < 0.2$ and also $\sin \pi/2 = 1 > 0.2$, the value of $x$ so that $\sin x = 0.2$ must be in between. Then we bisect this interval, check the value of sine at the new enpoint, and continue with the new subinterval where it changes sign. Here is the implementation.

In [8]:
function binary_search(f::Function,x0::Real,x1::Real,tolerance::Real=1e-15)
    counter = 0
    while abs(f(x1)) > tolerance
        x2 = (x0+x1)/2
        if f(x2)>=0 
            x1 = x2
        else
            x0 = x2
        end
        counter = counter + 1
        if counter > 1000
            println("We give up!")
            return
        end
    end
     println("We performed ", counter, " iterations.")
    return x1
end

binary_search (generic function with 2 methods)

In [9]:
x_binary = binary_search(f,0,pi/2)

We performed 47 iterations.


0.20135792079033127

With any of these methods, the computation is immediate in our computers. Still, using this silly bisection method took many more iterations. For a more complex equation, it might make a significant difference.

Let us now implement a method that combines the ideas of the secant method with the bisection method. That is, we always keep x0 and x1 being the endpoints of an interval where f changes sign. But we compute the endpoint of the next subinterval, not by bisection, but by the same computation as in the secant method. This is called *regula falsi* in the literature.

In [10]:
function regula_falsi(f::Function,x0::Real,x1::Real,tolerance::Real=1e-15)
    counter = 0
    while abs(f(x1)) > tolerance
        # Replace the following line with the right formula for x2.
        # It should be the same one that you used for the secant method.
        x2 = (x1*f(x0)-x0*f(x1))/(f(x0)-f(x1))
        if f(x2)>=0 
            x1 = x2
        else
            x0 = x2
        end
        counter = counter + 1
        if counter > 1000
            println("We give up!")
            return
        end
    end
    println("We performed ", counter, " iterations.")
    if abs(f(x1)) < abs(f(x0)) 
        return x1
    else
        return x0
    end
end

regula_falsi (generic function with 2 methods)

In [11]:
x_regula = regula_falsi(f,0,pi/2)

We performed 9 iterations.


0.20135792079033096

### Question
Find, approximately, the smallest positive value for `x0` so that when we execute `newton(f,df,x0)`, we get a different value.

### Question
Find, approximately, the smallest positive value for `x1` so that when we execute `secant(f,0,x1)`, we get a different value.

## Round-off errors

Let us do some experiments to see the effects of round-off errors by the computer. We know that if $\sin x = 0.2$, then we should also have $\sin (\pi-x) = 0$. Let us test it

In [12]:
sin(x_exact)-0.2, sin(pi-x_exact)-0.2

(0.0, 1.942890293094024e-16)

It might vary in different versions of Julia, and perhaps in different computers. The second number I got is slightly off. Subtracting two numbers of approximately the same size typically produces errors on a computer program. But the extent of these errors is difficult to predict.

It is probably a good idea to play around with these functions for a little while.

## Question

The followins is a relatively classical question. There is a circular field of grass of radius one. A cow is tied to a stake on the edge. How long should the rope be so that the cow is able to eat exactly half of the grass?

Since this is only the first week, I will help you out  (do not get used to this). After you make a drawing, whink for a little while, and write a few trigonometric functions, you get to the following formula. Here $r$ is the radius of the rope and $A$ is the area. Set $\alpha = 2 \arcsin(r/2)$ and $\beta = \pi-\alpha$. We get
$$A = \alpha + r^2 \frac \beta 2 - \frac r 2 \sqrt{ 4 - r^2 }$$

You can take this formula for granted to answer the question above.

In [13]:
function cow(r::Real)
    α = 2*asin(r/2)
    β = pi - α
    Area = α + r^2 * β / 2 - r/2 * sqrt(4-r^2)
    return Area - pi/2
end

secant(cow,0,1.5)
    

We performed 6 iterations.


1.1587284730181215

# Challenge

For the challenge question we use BigFloat. It is a data type that introduces floating point numbers of arbitrary precision.

We use an exageratedly large amout of bits to be sure. I set the precision to 300,000 binary digits. It is roughly equivalent to 100,000 decimal digits. Then we run the secant method with precision 1e-75000. Again, this is more than enough. Finally, we display the first 50000 digits.

In [15]:
setprecision(BigFloat,300000)

# We modify the function cow to use a precise approximation of pi.

function cow(r::BigFloat)
    α = 2*asin(r/2)
    β = pi - α
    Area = α + r^2 * β / 2 - r/2 * sqrt(4-r^2)
    return Area - BigFloat(pi)/2
end

cow (generic function with 2 methods)

In [16]:
res = secant(cow,big"0",big"1.5",big"1e-75000")
string(res)[1:50001]

We performed 23 iterations.


"1.1587284730181215178282335099335091496882922664920965118206958848206698025591960931993216107308604381759674951803405182901392508985190992648315448469064799414791317485292123564021555016323125555068614107819723352355017317306534399883035057982248155293321714183593309129" ⋯ 49462 bytes ⋯ "86729877434761822489457974226550332590927051903681731158958841675391333979958305643732400000774011375888851028693653661126374569511098037397077907915506707415699769272491545182240174642025420230132544136802326522292465882188302201561785746417952045108704274686197966156"

Note that if we try to use the binary_search method instead of the secant method, it takes too many iterations and it ends up failing.

In [17]:
res = binary_search(cow,big"0",big"1.5",big"1e-10000")

We give up!
